## Step 1️⃣: Install Dependencies

In [ ]:
# Install required packages
!pip install -q transformers torch librosa soundfile accelerate huggingface-hub vllm

print("✅ Dependencies installed!")

## Step 2️⃣: Clone Repository

In [ ]:
import os
from pathlib import Path

# Clone the project
!git clone https://github.com/hasipfaruk/Stock-Price-Extractor.git

# Navigate to project
os.chdir("Stock-Price-Extractor")
print("✅ Project cloned!")
print(f"📁 Current directory: {os.getcwd()}")

## Step 3️⃣: HuggingFace Authentication

In [ ]:
from huggingface_hub import login

# Set your token (replace with yours)
HF_TOKEN = "hf_YOUR_TOKEN_HERE"

if HF_TOKEN == "hf_YOUR_TOKEN_HERE":
    print("⚠️ IMPORTANT: Replace with your actual HuggingFace token!")
    print("📖 Get token from: https://huggingface.co/settings/tokens")
    print("1. Go to the link above")
    print("2. Create new token (read access is fine)")
    print("3. Copy and paste in the line above")
else:
    login(token=HF_TOKEN)
    print("✅ Authenticated with HuggingFace!")

## Step 4️⃣: Verify GPU

In [ ]:
import torch

print("=" * 70)
print("GPU STATUS")
print("=" * 70)

gpu_available = torch.cuda.is_available()
print(f"✅ GPU Available: {gpu_available}")

if gpu_available:
    print(f"📊 GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print("\n✨ GPU enabled - processing will be FAST!")
else:
    print("\n⚠️ GPU not available - will use CPU (slower)")
    print("💡 To enable GPU: Runtime → Change runtime type → GPU")

## Step 5️⃣: Upload Audio Files

In [ ]:
from google.colab import files
import os

# Create upload directory
os.makedirs('uploaded_audio', exist_ok=True)

print("📁 Upload audio files:")
print("1. Click 'Choose Files'")
print("2. Select multiple audio files (WAV, MP3, FLAC, M4A)")
print("3. Wait for upload to complete\n")

uploaded = files.upload()

print(f"\n✅ {len(uploaded)} files uploaded:")
for filename in uploaded.keys():
    file_path = f'uploaded_audio/{filename}'
    os.rename(filename, file_path)
    file_size = os.path.getsize(file_path) / (1024 * 1024)
    print(f"  📄 {filename} ({file_size:.1f} MB)")

## Step 6️⃣: Upload Extraction Prompt

In [ ]:
from google.colab import files

# Upload your prompt file
print("📝 Upload extraction prompt file:")
print("Click 'Choose Files' and select your prompt.txt file\n")

prompt_files = files.upload()

if prompt_files:
    prompt_filename = list(prompt_files.keys())[0]
    os.rename(prompt_filename, 'colab_prompt.txt')
    print(f"✅ Prompt uploaded: {prompt_filename}")
    
    # Show first 200 chars
    with open('colab_prompt.txt', 'r') as f:
        content = f.read()
    print(f"\n📖 Prompt preview ({len(content)} chars):")
    print(content[:200] + "...\n" if len(content) > 200 else content)
else:
    print("⚠️ No prompt file uploaded. Using default prompt.")
    # Create default prompt
    default_prompt = """Extract stock price information from the transcript.

Return JSON with these fields:
- index_name: Stock index name (e.g., \"S&P 500\")
- price: Current price
- change: Change in points
- change_percent: Percent change

Return ONLY valid JSON, no explanation."""
    
    with open('colab_prompt.txt', 'w') as f:
        f.write(default_prompt)
    print("✅ Default prompt created")

## Step 7️⃣: Import Functions

In [ ]:
import sys
from pathlib import Path

# Add project to path
project_path = Path.cwd()
sys.path.insert(0, str(project_path))

# Import functions
from app.models.transcribe import transcribe
from app.models.llm_extract import extract_with_long_prompt

print("✅ Functions imported successfully!")
print(f"  📍 transcribe() - ready")
print(f"  📍 extract_with_long_prompt() - ready")

## Step 8️⃣: Batch Processing (with Timing)

In [ ]:
import json
from pathlib import Path
import torch
import gc
import time

# Find all audio files
audio_files = sorted(list(Path('uploaded_audio').glob('*')))
audio_files = [f for f in audio_files if f.suffix.lower() in ['.wav', '.mp3', '.flac', '.m4a', '.ogg']]

print(f"📁 Found {len(audio_files)} audio files\n")

if len(audio_files) == 0:
    print("❌ No audio files to process")
else:
    all_results = {}
    batch_start_time = time.time()
    
    for i, audio_file in enumerate(audio_files, 1):
        filename = audio_file.name
        file_start_time = time.time()
        print(f"[{i}/{len(audio_files)}] Processing {filename}...", end=' ', flush=True)
        
        try:
            # Clear memory
            torch.cuda.empty_cache()
            gc.collect()
            
            # Transcribe
            trans_start = time.time()
            result = transcribe(str(audio_file))
            trans_duration = time.time() - trans_start
            
            transcript = result.get('result') if isinstance(result, dict) else result
            trans_time = result.get('time', trans_duration) if isinstance(result, dict) else trans_duration
            
            # Extract
            extract_start = time.time()
            extraction = extract_with_long_prompt(transcript, prompt_file='colab_prompt.txt')
            extract_duration = time.time() - extract_start
            
            file_total = time.time() - file_start_time
            
            all_results[filename] = {
                "status": "success",
                "data": extraction,
                "timing": {
                    "transcription_s": round(trans_time, 3),
                    "extraction_s": round(extract_duration, 3),
                    "total_s": round(file_total, 3)
                }
            }
            print(f"✅ ({file_total:.2f}s)")
            
        except Exception as e:
            file_total = time.time() - file_start_time
            all_results[filename] = {
                "status": "error",
                "error": str(e)[:100],
                "timing": {
                    "total_s": round(file_total, 3)
                }
            }
            print(f"❌ ({file_total:.2f}s)")
    
    # Save results
    with open('batch_results.json', 'w') as f:
        json.dump(all_results, f, indent=2)
    
    batch_total_time = time.time() - batch_start_time
    print(f"\n✅ All results saved to: batch_results.json")
    
    # Summary with timing
    success = sum(1 for r in all_results.values() if r["status"] == "success")
    failed = len(all_results) - success
    avg_time = sum(r.get("timing", {}).get("total_s", 0) for r in all_results.values() if r["status"] == "success") / max(success, 1)
    
    print(f"\n📊 Summary:")
    print(f"  ✅ Successful: {success}/{len(all_results)}")
    print(f"  ❌ Failed: {failed}/{len(all_results)}")
    print(f"  ⏱️ Batch Total: {batch_total_time:.2f}s")
    print(f"  ⏱️ Average per file: {avg_time:.2f}s")
    
    # Show sample results with timing
    if success > 0:
        print(f"\n📈 Sample Results:")
        for filename, result in list(all_results.items())[:3]:
            if result["status"] == "success":
                print(f"\n  {filename}:")
                data = result["data"]
                timing = result.get("timing", {})
                print(f"    Index: {data.get('index_name')}")
                print(f"    Price: {data.get('price')}")
                print(f"    Change: {data.get('change')} ({data.get('change_percent')})")
                print(f"    🎤 Transcription: {timing.get('transcription_s', 0):.3f}s")
                print(f"    🤖 Extraction: {timing.get('extraction_s', 0):.3f}s")
                print(f"    ⏱️ Total: {timing.get('total_s', 0):.3f}s")

## Step 9️⃣: Download Results

In [ ]:
from google.colab import files
from pathlib import Path

print("📥 Download your results:\n")

# Download batch results if exists
if Path('batch_results.json').exists():
    print("1. Downloading batch_results.json (with timing data)...")
    files.download('batch_results.json')
    print("   ✅ Downloaded!")

print("\n✅ All files ready for download!")
print("\nCheck the 'Files' tab (left panel) to download any files.")
print("\n📊 Results include:")
print("  - Index name, price, change, change%")
print("  - 🎤 Transcription time (seconds)")
print("  - 🤖 Extraction time (seconds)")
print("  - ⏱️ Total time per file (seconds)")

## 🎉 Complete!

Your stock price extraction is done! 🚀

**⏱️ What You Got:**
1. ✅ Stock prices extracted from audio
2. 🎤 Transcription time for each file
3. 🤖 LLM extraction time for each file
4. ⏱️ Total processing time per file

**Next steps:**
1. Download `batch_results.json` from the Files panel
2. Analyze timing data to optimize performance
3. Import results into your spreadsheet or database

**For more info:**
- See `README.md` for general documentation
- See `TIMING_FEATURES.md` for timing guide
- See `USAGE.md` for usage examples